In [134]:
CELSIUS_STR = 'celsius'
FAHRENHEIT_STR = 'fahrenheit'

In [135]:
import re
from typing import Dict

def parse_temperature(input_str: str) -> tuple[float, str]:
	scales = {
		CELSIUS_STR: ['c', 'celsius', '°c'],
		FAHRENHEIT_STR: ['f', 'fahrenheit', '°f'],
	}

	value_match = re.search(r'([-+]?\d+(\.\d+)?)', input_str)
	if value_match:
		value = float(value_match.group(1))
	else:
		raise ValueError("Temperature value not found")

	scale = None

	for key, values in scales.items():
		for val in values:
			if val.lower() in input_str.lower():
				scale = key
				break
		if scale:
			break

	if not scale:
		scale = CELSIUS_STR

	return (value, scale)


In [136]:
URL = 'https://www.w3schools.com/xml/tempconvert.asmx'

In [137]:
def converter(value: float, scale):
	import requests

	body = f"""<?xml version="1.0" encoding="utf-8"?>
<soap12:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap12="http://www.w3.org/2003/05/soap-envelope">
  <soap12:Body>
  {
	  scale == FAHRENHEIT_STR and 
	  f"""<FahrenheitToCelsius xmlns="https://www.w3schools.com/xml/">
	  		<Fahrenheit>
				{value}
			</Fahrenheit>
		</FahrenheitToCelsius>""" 
		or 
	  f"""<CelsiusToFahrenheit xmlns="https://www.w3schools.com/xml/">
	  		<Celsius>
				{value}
			</Celsius>
		</CelsiusToFahrenheit>"""
  }
  </soap12:Body>
</soap12:Envelope>"""

	headers = {
		'Content-Type': 'application/soap+xml; charset=utf-8',
		'Host': 'www.w3schools.com',
		'User-Agent': 'Mozilla/5.0',
		'Content-Length': str(len(body)),
	}

	response = requests.post(URL, headers=headers, data=body)

	match = re.search(r'<(FahrenheitToCelsius|CelsiusToFahrenheit)Result>(.*?)</\1Result>', response.text)
	return float(match.group(2))



In [138]:
(value, scale) = parse_temperature(input("Enter temperature and scale (f or c or variations thereof): "))


In [139]:
converted = converter(value, scale)


In [140]:
print(f"{value:.2f} {scale.upper()[0]} is {converted:.2f}° {'F' if scale == CELSIUS_STR else 'C'}")

40.00 C is 104.00° F
